# Effect of $\epsilon$ on Average Treatment Effect on LaLonde

In [1]:
import copy

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch

from misc.agm import calibrateAnalyticGaussianMechanism

%matplotlib inline

# set random seed
np.random.seed(1)
torch.manual_seed(1)

In [2]:
# no. experiments, no. samples for fitting, no. samples for estimating, no. of draws of z
ne = 1000
nf = 500
nt = 200
nd = 1

# privacy parameters
epses = [0.2, 0.4, 0.6, 0.8, 0.99]
delta = 1. / (nf ** 2)

# regularisation coefficient
reg_co = 0.1

## Preprocess LaLonde data

In [3]:
# read data and remove index and rename last column
lalonde_df = (
    pd.read_csv('data/LaLonde-CBPS/lalonde.csv')
    .drop('Unnamed: 0', 1)
    .rename(columns={'re74.miss': 're74_miss'})
)

# remove points not in original LaLonde dataset and drop exper
lalonde_df = (
    lalonde_df[lalonde_df.exper == 1]
#     lalonde_df[lalonde_df.re74_miss == 0]
    .drop(['exper'], 1)
#     .drop(['exper', 're74_miss'], 1)
)

# change column positions
cols = list(lalonde_df.columns)
cols = cols[:-2] + [cols[-1]] + [cols[-2]]
lalonde_df = lalonde_df[cols]

# use dict structure
lalonde = {}

# get X, T, Y
# restrict X to ||x||_2 \leq 1 to fit assumption
X = torch.tensor(lalonde_df.iloc[:, 1:-1].values, dtype=torch.float64)
lalonde['x'] = X / X.norm(dim=1).max()
lalonde['t'] = torch.tensor(lalonde_df['treat'].values, dtype=torch.float64)
lalonde['y'] = torch.tensor(lalonde_df['re78'].values, dtype=torch.float64)

In [4]:
# get no. experiments and dim
_, d = lalonde['x'].shape

In [5]:
# generate X_test, T_test, Y_test through subsampling
Y_test, X_test = [], []
T_test = torch.stack([torch.cat(
    [torch.ones(int(nt/2), dtype=torch.float64), torch.zeros(int(nt/2), dtype=torch.float64)])] * ne
)

Y_train, X_train = [], []
T_train = torch.stack([torch.cat(
    [torch.ones(int(nf/2), dtype=torch.float64), torch.zeros(int(nf/2), dtype=torch.float64)])] * ne
)

for i in range(ne):
    # get indices for t=1 and t=0
    t1_idx = lalonde['t'].nonzero().squeeze().numpy()
    t0_idx = (1 - lalonde['t']).nonzero().squeeze().numpy()
                 
    # subsample without replacement nt indices, nt/2 for t=1 and nt/2 for t=0
    sam_t1, sam_t0 = np.random.choice(t1_idx, int(nt/2), replace=False), np.random.choice(t0_idx, int(nt/2), replace=False)
    sam_idx = np.hstack([sam_t1, sam_t0])

    # subsample with replacement nf indices, nf/2 for t=1 and nf/2 for t=0
    unsam_t1, unsam_t0 = np.setxor1d(t1_idx, sam_t1), np.setxor1d(t0_idx, sam_t0)
    unsam_idx = np.hstack([np.random.choice(unsam_t1, int(nf/2), replace=True), np.random.choice(unsam_t0, int(nf/2), replace=True)])
    
    Y_test.append(lalonde['y'][sam_idx])
    X_test.append(lalonde['x'][sam_idx, :])

    Y_train.append(lalonde['y'][unsam_idx])
    X_train.append(lalonde['x'][unsam_idx, :])

# convert to torch tensors 
Y_test, X_test = torch.stack(Y_test), torch.stack(X_test)
Y_train, X_train = torch.stack(Y_train), torch.stack(X_train)

# permute data
# permute indices
perm_test = torch.stack([torch.randperm(nt) for i in range(ne)])
perm_train = torch.stack([torch.randperm(nf) for i in range(ne)])

# create auxiliary indices
idx = torch.arange(ne)[:, None]

# permute X, T, Y
X_all = torch.cat([X_train[idx, perm_train], X_test[idx, perm_test]], 1) 
T_all = torch.cat([T_train[idx, perm_train], T_test[idx, perm_test]], 1) 
Y_all = torch.cat([Y_train[idx, perm_train], Y_test[idx, perm_test]], 1) 

## Define model and method

In [6]:
class Log_Reg(torch.nn.Module):
    '''
    Logistic Regression
    '''
    def __init__(self, D_in, D_out):
        super(Log_Reg, self).__init__()
        self.linear = torch.nn.Linear(D_in, D_out, bias=False)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

In [7]:
def IPTW_PPS(X, T, Y, epses, delta, reg_co, nd, nf):
    '''
    average treatment effect with inverse probability of treatment weighting using private propensity scores
    '''
    # get # experiments, # samples, # dimensions
    ne, ns, dim = X.shape

    # sgd step size
    step_size = 0.01

    ################
    # process data #
    ################

    # get Y0 and Y1
    Y0 = Y * (1 - T)
    Y1 = Y * T

    # split data
    # get splits
    fit_split = nf
    est_split = ns - nf

    # permute indices
    perm = torch.stack(
        [torch.randperm(ns) for i in range(ne)]
    )

    # create auxiliary indices
    idx = torch.arange(ne)[:, None]

    # split X into fit, estimate splits
    X_s0 = X[:, :fit_split]
    X_s1 = X[:, fit_split:]

    # expand dim of T to allow multiplication with X
    T_ex_dim = T.reshape(ne, ns, 1)

    # split X0 and X1 into fit, estimate splits
    X0_s1 = (X * (1 - T_ex_dim))[:, fit_split:]
    X1_s1 = (X * T_ex_dim)[:, fit_split:]

    # precompute ||X0_sl||_2^2 and ||X1_sl||_2^2
    X0_s1_2 = X0_s1.norm(dim=2) ** 2
    X1_s1_2 = X1_s1.norm(dim=2) ** 2

    # add x_i to x_j for X0_plus_X0_s1 and X1_plus_X1_s1
    X0_plus_X0_s1 = X0_s1.reshape(
        ne, est_split, 1, dim
    ) + X0_s1.reshape(ne, 1, est_split, dim)
    X1_plus_X1_s1 = X1_s1.reshape(
        ne, est_split, 1, dim
    ) + X1_s1.reshape(ne, 1, est_split, dim)

    # get norms squared for X0_plus_X0_s1 and X1_plus_X1_sl
    X0_plus_X0_sl_norm_2 = X0_plus_X0_s1.norm(dim=-1) ** 2
    X1_plus_X1_sl_norm_2 = X1_plus_X1_s1.norm(dim=-1) ** 2

    # split T into fit, estimate splits
    T_s0 = T[:, :fit_split]
    T_s1 = T[:, fit_split:]

    # split Y0 and Y1 into fit, estimate splits
    Y0_s0 = Y0[:, :fit_split]
    Y1_s0 = Y1[:, :fit_split]

    Y0_s1 = Y0[:, fit_split:]
    Y1_s1 = Y1[:, fit_split:]

    # mult y_i to y_j for Y0_times_Y0_s1 and Y1_times_Y1_s1
    Y0_times_Y0_s1 = Y0_s1.reshape(
        ne, est_split, 1
    ) * Y0_s1.reshape(ne, 1, est_split)
    Y1_times_Y1_s1 = Y1_s1.reshape(
        ne, est_split, 1
    ) * Y1_s1.reshape(ne, 1, est_split)

    # reshape estimate splits for later
    Y0_s1 = Y0_s1.reshape(ne, 1, est_split)
    Y1_s1 = Y1_s1.reshape(ne, 1, est_split)

    ##############
    # fit models #
    ##############

    # instantiate ne different models
    models = [Log_Reg(dim, 1) for i in range(ne)]
    # set model parameters to float64
    [model.double() for model in models]

    # define loss (binary cross entropy)
    loss = torch.nn.BCELoss()

    # define optimisers
    optimisers = [
        torch.optim.SGD(
            models[i].parameters(),
            lr=step_size,
            weight_decay=reg_co,
        )
        for i in range(ne)
    ]

    # train models
    for t in range(1000):
        preds = [
            models[i](X_s0[i]).squeeze() for i in range(ne)
        ]
        losses = [
            loss(preds[i], T_s0[i]) for i in range(ne)
        ]
        [opt.zero_grad for opt in optimisers]
        [loss.backward() for loss in losses]
        [opt.step() for opt in optimisers]

    #############################
    # estimate treatment effect #
    #############################

    # initialise pi_hat dictionaries
    pi_hats = {}
    pi_hats_analytic = {}

    # get estimated propensity scores
    pi_hats[0] = torch.stack(
        [models[i](X_s1[i]).squeeze() for i in range(ne)]
    )

    # perturb model and get relevant quantities
    for eps in epses:
        # define sigma
        s_a = 2.0 / (fit_split * reg_co)

        # gaussian mechanism
        sigma = np.sqrt(
            2 * np.log(1.25 / delta) + 1e-10
        ) * (s_a / eps)
        sigma_2 = sigma ** 2

        # # analytic gaussian mechanism
        # sigma = calibrateAnalyticGaussianMechanism(eps, delta, s_a)
        # sigma_2 = sigma ** 2

        # define noise distribution
        noise_dist = torch.distributions.normal.Normal(
            torch.tensor([0.0], dtype=torch.float64),
            torch.tensor([sigma], dtype=torch.float64),
        )

        # draw noise vectors
        noise_vecs = noise_dist.sample(
            (ne, nd, dim)
        ).reshape(ne, nd, dim)

        # create temp models
        models_ = [copy.deepcopy(models) for i in range(nd)]

        # \hat{\w}^\top Xs
        w_T_X0_s1 = []
        w_T_X0_plus_X0_s1 = []
        w_T_X1_s1 = []
        w_T_X1_plus_X1_s1 = []

        # initialise list for privatised estimated propensity scores
        pi_hats[eps] = []

        # perturb weights with noise vectors
        for i in range(ne):
            w_T_X0_s1.append(
                torch.einsum(
                    'ij,kj-> i',
                    X0_s1[i],
                    models[i].linear.weight,
                )
            )
            w_T_X0_plus_X0_s1.append(
                torch.einsum(
                    'ijk,lk-> ij',
                    X0_plus_X0_s1[i],
                    models[i].linear.weight,
                )
            )
            w_T_X1_s1.append(
                torch.einsum(
                    'ij,kj-> i',
                    X1_s1[i],
                    models[i].linear.weight,
                )
            )
            w_T_X1_plus_X1_s1.append(
                torch.einsum(
                    'ijk,lk-> ij',
                    X1_plus_X1_s1[i],
                    models[i].linear.weight,
                )
            )
            for j in range(nd):
                model_temp = models_[j][i]
                model_temp.linear.weight.data.add_(
                    noise_vecs[i, j, :]
                )
                pi_hats[eps].append(
                    model_temp(X_s1[i]).squeeze()
                )
                
        # reshape stacked privatised estimated propensity scores as ne * nd
        pi_hats[eps] = torch.stack(pi_hats[eps]).reshape(ne, nd, est_split)

        # precompute sigma^2 ||x||_2^2 and \w^\top Xs
        pi_hats_analytic[eps] = {'sigma_2_X0_s1_2': sigma_2 * X0_s1_2}
        pi_hats_analytic[eps]['sigma_2_X1_s1_2'] = sigma_2 * X1_s1_2
        pi_hats_analytic[eps]['sigma_2_X0_plus_X0_s1'] = sigma_2 * X0_plus_X0_sl_norm_2
        pi_hats_analytic[eps]['sigma_2_X1_plus_X1_s1'] = sigma_2 * X1_plus_X1_sl_norm_2
        pi_hats_analytic[eps]['w_T_X0_s1'] = torch.stack(w_T_X0_s1)
        pi_hats_analytic[eps]['w_T_X0_plus_X0_s1'] = torch.stack(w_T_X0_plus_X0_s1)
        pi_hats_analytic[eps]['w_T_X1_s1'] = torch.stack(w_T_X1_s1)
        pi_hats_analytic[eps]['w_T_X1_plus_X1_s1'] = torch.stack(w_T_X1_plus_X1_s1)

    # get treatment effects
    # empirical means and (std of means) of ERM + private ERM
    te_hats = {'means': [], 'stds': []}
    # analytic means and (std of means + mean of stds) of private ERM
    te_hats_analytic = {'means': [], 'stds': []}

    for key in pi_hats.keys():
        if key != 0:
            # empirical estimates
            # reduce_mean from (ne, nd, est_split) tensor to (ne * nd, 1) matrix
            te_hats_ = torch.mean(
                Y1_s1 / pi_hats[key] - Y0_s1 / (1 - pi_hats[key]),
                2,
            )
            # analytic estimates
            # expectation and variance of mu_0
            rand_mu_0 = Y0_s1.squeeze() * torch.exp(
                pi_hats_analytic[key]['w_T_X0_s1'] + pi_hats_analytic[key]['sigma_2_X0_s1_2'] / 2
            )
            E_mu_0 = torch.mean(
                Y0_s1.squeeze() + rand_mu_0, 
                [1]
            )
            mu_X0_s1_2 = Y0_times_Y0_s1 * torch.exp(
                pi_hats_analytic[key]['w_T_X0_plus_X0_s1'] + pi_hats_analytic[key]['sigma_2_X0_plus_X0_s1'] / 2
            )
            mu_X0_s1_mu_X0_s1 = rand_mu_0.reshape(ne, 1, est_split) * rand_mu_0.reshape(ne, est_split, 1)
            var_mu_0 = torch.mean(
                mu_X0_s1_2 - mu_X0_s1_mu_X0_s1, 
                [1, 2]
            )
            # expectation and variance of mu_1
            rand_mu_1 = Y1_s1.squeeze() * torch.exp(
                - (pi_hats_analytic[key]['w_T_X1_s1'] + pi_hats_analytic[key]['sigma_2_X1_s1_2'] / 2)
            )
            E_mu_1 = torch.mean(
                Y1_s1.squeeze() + rand_mu_1, 
                [1]
            )
            mu_X1_s1_2 = Y1_times_Y1_s1 * torch.exp(
                - (pi_hats_analytic[key]['w_T_X1_plus_X1_s1'] + pi_hats_analytic[key]['sigma_2_X1_plus_X1_s1'] / 2)
            )
            mu_X1_s1_mu_X1_s1 = rand_mu_1.reshape(ne, 1, est_split) * rand_mu_1.reshape(ne, est_split, 1)
            var_mu_1 = torch.mean(
                mu_X1_s1_2 - mu_X1_s1_mu_X1_s1, 
                [1, 2]
            )
            # expectation and variance of te_hats
            te_hats_analytic_mu = E_mu_1 - E_mu_0
            te_hats_analytic_std = torch.sqrt(
                var_mu_1 + var_mu_0
            )
            te_hats_analytic['means'].append(
                te_hats_analytic_mu.detach().numpy()
            )
            te_hats_analytic['stds'].append(
                te_hats_analytic_std.detach().numpy()
            )
        else:
            # empirical estimate for noiseless case
            # reduce_mean from (ne, est_split) tensor to (ne , 1) matrix
            te_hats_ = torch.mean(
                Y1_s1.squeeze() / pi_hats[key] - Y0_s1.squeeze() / (1 - pi_hats[key]),
                1,
            ).reshape(ne, 1)
        te_hats['means'].append(
            [
                te_hats_[i].mean().detach().numpy()
                for i in range(ne)
            ]
        )
        te_hats['stds'].append(
            [
                te_hats_[i].std().detach().numpy()
                for i in range(ne)
            ]
        )

    te_hats['means'] = np.array(te_hats['means'])
    te_hats['stds'] = np.array(te_hats['stds'])
    te_hats_analytic['means'] = np.array(
        te_hats_analytic['means']
    )
    te_hats_analytic['stds'] = np.array(
        te_hats_analytic['stds']
    )

    return te_hats, te_hats_analytic

## Run method and print results

In [8]:
te_hats, te_hats_analytic = IPTW_PPS(X_all, T_all, Y_all, epses, delta, reg_co, nd, nf)

In [9]:
means = [np.mean(i) for i in te_hats['means']]
for i in range(len(means)):
    if i == 0:
        print('The mean ATE for no epsilon is {}'.format(means[i]))
    else:
        print('The mean ATE for epsilon = {} is {}'.format(epses[i-1], means[i]))                        

The mean ATE for no epsilon is 902.1086545150501
The mean ATE for epsilon = 0.2 is 872.2163852736154
The mean ATE for epsilon = 0.4 is 892.2209852702304
The mean ATE for epsilon = 0.6 is 904.9505716458677
The mean ATE for epsilon = 0.8 is 898.8150398526142
The mean ATE for epsilon = 0.99 is 904.0730387357378


In [10]:
sgn_tau_hat = np.sign(te_hats['means'][0])
    
# compute probabilities
probs = [sum(np.sign(i) != sgn_tau_hat) / ne for i in te_hats['means'][1:]]

for i in range(len(epses)):
    print('The probability of signs being flipped for epsilon = {} is {}'.format(epses[i], probs[i]))     

The probability of signs being flipped for epsilon = 0.2 is 0.143
The probability of signs being flipped for epsilon = 0.4 is 0.068
The probability of signs being flipped for epsilon = 0.6 is 0.05
The probability of signs being flipped for epsilon = 0.8 is 0.03
The probability of signs being flipped for epsilon = 0.99 is 0.028
